In [1]:
import importlib
import keys
importlib.reload(keys)
import json
import pprint
keychain = keys.keychain

pp = pprint.PrettyPrinter(indent=2)

client_id = keychain['youtube']['client_id']
scope = keychain['youtube']['scope']
redirect = keychain['youtube']['redirect_uris'][0]
print(client_id[:20], scope, redirect)

126920892433-e58m7pn ['https://www.googleapis.com/auth/youtube.readonly'] urn:ietf:wg:oauth:2.0:oob


In [2]:
from requests_oauthlib import OAuth2Session

In [3]:
session = OAuth2Session(client_id, scope=scope, redirect_uri=redirect)
auth_url, state = session.authorization_url(keychain['youtube']['auth_uri'],access_type='offline',include_granted_scopes='true')
print(auth_url)

https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=126920892433-e58m7pn87600743drvlsjsanlcsus2av.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=Vlxtl2oHVOPlCiC8YdmXRh269FoTp0&access_type=offline&include_granted_scopes=true


In [4]:
import webbrowser
webbrowser.open(auth_url)

True

In [8]:
code = '4/BKsviDml-GvvgllvlX3WWgr1t41STLklP91cF2w3R9o'

In [9]:
token = session.fetch_token(keychain['youtube']['token_uri'],
                            code=code,
                            client_secret=keychain['youtube']['client_secret'])  

In [ ]:
print(token)

In [59]:
import json
with open('token.json', 'w') as outfile:
    json.dump(token, outfile)

In [60]:
token=open('token.json',mode='r')
token=token.read()
token

'{"access_token": "ya29.GlsbBWJVuN5hKsSXIggsN2d_swCNCuXWyg5El-9RDyTno40_v3Gh1rFjqigb3ecGD808HkRkwDMjWz7I_zLj1rWy7qyMIQ98NaugBhn6pGQCMtOU5WmijF_DSFpZ", "expires_in": 3600, "token_type": "Bearer", "expires_at": 1512619990.0627892, "refresh_token": "1/lJ_us6eBCLri1jPS093V-E4qqpra0RYQ5UUMF-hY5m8"}'

In [61]:
token=json.loads(token)
token

{'access_token': 'ya29.GlsbBWJVuN5hKsSXIggsN2d_swCNCuXWyg5El-9RDyTno40_v3Gh1rFjqigb3ecGD808HkRkwDMjWz7I_zLj1rWy7qyMIQ98NaugBhn6pGQCMtOU5WmijF_DSFpZ',
 'expires_at': 1512619990.0627892,
 'expires_in': 3600,
 'refresh_token': '1/lJ_us6eBCLri1jPS093V-E4qqpra0RYQ5UUMF-hY5m8',
 'token_type': 'Bearer'}

In [62]:
importlib.reload(keys)
keychain = keys.keychain
keychain['youtube']

{'auth_provider_x509_cert_url': 'https://www.googleapis.com/oauth2/v1/certs',
 'auth_uri': 'https://accounts.google.com/o/oauth2/auth',
 'client_id': '126920892433-e58m7pn87600743drvlsjsanlcsus2av.apps.googleusercontent.com',
 'client_secret': 'flFfvozgUappY1MIRCPE5Cxc',
 'project_id': 'project5-187402',
 'redirect_uris': ['urn:ietf:wg:oauth:2.0:oob', 'http://localhost'],
 'scope': ['https://www.googleapis.com/auth/youtube.readonly'],
 'token_uri': 'https://accounts.google.com/o/oauth2/token'}

In [63]:
import requests
import pandas as pd
from xml.etree import ElementTree as ET
import io
D = {}
D['access_token'] = token['access_token']

In [64]:
data = {'videoId':[], 'channelId':[],'categoryId':[] ,'channelTitle':[], 'publishedAt':[]}
def getData(data,vid):
    for item in vid:
        data['videoId'].append(item['id'])
        data['channelId'].append(item['snippet']['channelId'])
        data['categoryId'].append(item['snippet']['categoryId'])
        data['channelTitle'].append(item['snippet']['channelTitle'])
        data['publishedAt'].append(item['snippet']['publishedAt'])

In [65]:
url ='https://www.googleapis.com/youtube/v3/videos'
D['part'] = ['snippet,contentDetails,statistics']
D['maxResults'] = '50'
D['chart'] ='mostPopular'
popular_vid = requests.get(url, params = D)
popular_vid = popular_vid.json()

In [66]:
page = []
data = {'videoId':[], 'channelId':[],'categoryId':[] ,'channelTitle':[], 'publishedAt':[]}
for i in range(20):
    if 'nextPageToken' not in popular_vid:
        nex_page = 'CAUQAA'
    else:
        next_page = popular_vid['nextPageToken']
    page.append(next_page)
    D['pageToken'] = next_page
    popular_vid = requests.get(url, params = D)
    popular_vid = popular_vid.json()
    getData(data,popular_vid['items'])
df= pd.DataFrame(data)
df[:10]

,categoryId,channelId,channelTitle,publishedAt,videoId
0,24,UCIRiWCPZoUyZDbydIqitHtQ,Mike Boyd,2017-12-04T12:00:03.000Z,X6iJ0hPpGec
1,10,UCZU9T1ceaOgwfLRq7OKFU4Q,Linkin Park,2017-12-04T15:00:48.000Z,sjN-NGsRg9g
2,24,UC4PooiX37Pld1T8J5SYT-SQ,Good Mythical Morning,2017-12-04T11:00:00.000Z,-C5q3oir8gA
3,22,UC8C3teVwucjcg7skFUHtPEw,FILMWEBTV,2017-12-05T15:10:37.000Z,XSP5yHZN1I4
4,10,UCWg7ZcouKaf4TQc_too6kGQ,KISS FM UK,2017-12-05T14:00:49.000Z,u5Jw7D-c3II
5,26,UCrIIH7yHQugaUuoTZDoEUuQ,Tanya Burr,2017-12-05T15:30:02.000Z,bqbWMgkvedI
6,17,UCOBlh9E4bgmCWrDhuQGEkQA,Syracuse Orange,2017-12-06T05:22:41.000Z,h4kv0iKgiks
7,17,UCWJ2lWNubArHWmf3FIHbfcQ,NBA,2017-12-06T06:36:23.000Z,j5cSKF5FzD0
8,25,UCLXo7UDZvByw2ixzpQCufnA,Vox,2017-12-04T14:09:43.000Z,a-4_6bThk2E
9,23,UCjD2KyAEm84yVH8cTilID7Q,Cracked,2017-12-04T17:00:06.000Z,xi8u9DcC0_Q


In [39]:
url = 'https://www.googleapis.com/youtube/v3/videoCategories'
C = {}
C['access_token'] = token['access_token']
C['part'] = 'snippet'
C['regionCode'] = 'US'
category_vid = requests.get(url, params = C)
category_vid = category_vid.json()

In [34]:
data_category = {'id':[],'title':[]}
def categoryTable(data,json):
    for item in json:
        data['id'].append(item['id'])
        data['title'].append(item['snippet']['title'])
categoryTable(data_category,category_vid['items'])
df1 = pd.DataFrame(data_category)
df1[:10]

,id,title
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music
3,15,Pets & Animals
4,17,Sports
5,18,Short Movies
6,19,Travel & Events
7,20,Gaming
8,21,Videoblogging
9,22,People & Blogs


In [38]:
result = pd.merge(df, df1, left_on='categoryId', right_on='id')
result.to_csv('result.csv')

In [57]:
refresh_url = keychain['youtube']['token_uri']
token = session.refresh_token(refresh_url, 
                             client_id=keychain['youtube']['client_id'],
                             client_secret=keychain['youtube']['client_secret'])

In [58]:
print(token)

{'access_token': 'ya29.GlsbBWJVuN5hKsSXIggsN2d_swCNCuXWyg5El-9RDyTno40_v3Gh1rFjqigb3ecGD808HkRkwDMjWz7I_zLj1rWy7qyMIQ98NaugBhn6pGQCMtOU5WmijF_DSFpZ', 'expires_in': 3600, 'token_type': 'Bearer', 'expires_at': 1512619990.0627892, 'refresh_token': '1/lJ_us6eBCLri1jPS093V-E4qqpra0RYQ5UUMF-hY5m8'}
